# 🧹 Limpeza de Coordenadas - Pernambuco
Este notebook remove pontos redundantes que formam retas (inclusive inclinadas) a partir de um arquivo JSON com coordenadas.

In [ ]:
# 📦 Importar bibliotecas
import json
import numpy as np

In [ ]:
pathDados = "./output_ne_coordinates/Pernambuco.json"

In [ ]:
# ⚙️ Função para detectar retas (horizontal, vertical ou inclinada)
def is_redundant_straight(p1, p2, p3, angle_threshold=3.0):
    """Detecta se três pontos estão praticamente alinhados (reta)."""
    a = np.array([p1["lng"], p1["lat"]])
    b = np.array([p2["lng"], p2["lat"]])
    c = np.array([p3["lng"], p3["lat"]])

    ab = b - a
    bc = c - b

    ab_len = np.linalg.norm(ab)
    bc_len = np.linalg.norm(bc)

    if ab_len == 0 or bc_len == 0:
        return True  # ponto repetido

    cosine = np.dot(ab, bc) / (ab_len * bc_len)
    angle = np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

    return abs(angle) < angle_threshold or abs(angle - 180) < angle_threshold

In [ ]:
# 📥 Carregar o arquivo com coordenadas
with open(pathDados, "r", encoding="utf-8") as f:
    coords = json.load(f)

print(f"Total de coordenadas originais: {len(coords)}")

In [ ]:
# 🧼 Remover pontos que formam retas puras
final_coords = []
i = 0
while i < len(coords) - 2:
    p1, p2, p3 = coords[i], coords[i+1], coords[i+2]
    if is_redundant_straight(p1, p2, p3):
        final_coords.append(p1)
        i += 2  # pula ponto do meio
    else:
        final_coords.append(p1)
        i += 1

# Adiciona os dois últimos pontos
final_coords.extend(coords[-2:])
print(f"Total após remoção de retas: {len(final_coords)}")

In [ ]:
# 💾 Salvar o resultado
with open("Pernambuco_sem_qualquer_reta.json", "w", encoding="utf-8") as f:
    json.dump(final_coords, f, indent=2, ensure_ascii=False)

print("Arquivo salvo como 'Pernambuco_sem_qualquer_reta.json'")